In [8]:
import json
import pandas as pd
from FlagEmbedding import BGEM3FlagModel
import os
import json
import numpy as np

# Download and preprocess dataset

In [3]:
from datasets import load_dataset

ds = load_dataset("wangrui6/Zhihu-KOL")

In [4]:
data_dict = {}
data_dict['question'] = ds['train']['INSTRUCTION']
data_dict['answer'] = ds['train']['RESPONSE']
for key in json.loads(ds['train']['METADATA'][0]).keys():
    data_dict[key] = [json.loads(x)[key] for x in ds['train']['METADATA']]

In [5]:
df_train = pd.DataFrame(data_dict)
print(df_train.shape[0])
df_train.head()

1006218


,question,answer,question_id,answer_id,url,upvotes,answer_creation_time
0,怎么说服男朋友买烤箱？,emmmmm，首先想说的是，我买厨房用品一般是不用「说服」的，只是在厨房堆的满满当当的情况下...,357137111.0,9.143328e+08,https://www.zhihu.com/question/357137111/answe...,赞同 15,2019-11-28T12:01:22.000Z
1,航天从业者是如何看待电视剧《你是我的荣耀》的？,难得有个关于航天的剧，职场情节悬不悬浮，航天设定和细节走不走心？带着放大镜看了前18集，...,475169837.0,2.053313e+09,https://www.zhihu.com/question/475169837/answe...,赞同 4432,2021-08-11T07:26:08.000Z
2,如何看待PayPal正式进入中国？,PayPal不仅是美国支付巨头，也是国际支付巨头，目前已开拓全球200多个市场，美国以外的市...,348551037.0,8.661034e+08,https://www.zhihu.com/question/348551037/answe...,赞同 127,2019-10-22T09:11:15.000Z
3,中金公司交易员月薪八万五是如何做到的？,1、首先，考虑到这位交易员的工作经验，月薪八万五的表述是不正确的：其实是一年的全部薪酬除以1...,545938899.0,2.602364e+09,https://www.zhihu.com/question/545938899/answe...,赞同 450,2022-07-31T13:29:04.000Z
4,摇滚乐（金属）给你们带来了什么？,"ㄟ( ▔, ▔ )ㄏ哪里带来了什么东西啊，除了找到热爱的东西，也失去了很多。听重型现场像疯子...",361437216.0,1.073541e+09,https://www.zhihu.com/question/361437216/answe...,赞同 5,2020-03-12T05:49:28.000Z


In [5]:
len(set(df_train['question_id']))

159007

In [6]:
len(set(df_train['answer_id']))

1006218

In [33]:
def clean_upvotes(upvote_str):
    upvote_str = upvote_str.replace('赞同', '').strip()
    if len(upvote_str) > 0:
        if "万" in upvote_str:
            upvote_str = upvote_str.replace('万', '').strip()
            return int(float(upvote_str) * 10000)
        else:
            return int(upvote_str)
    else:
        return 0


df_train['upvotes'] = df_train['upvotes'].apply(clean_upvotes)
df_train['answer_id'] = df_train['answer_id'].astype('int64')
df_train['question_id'] = df_train['question_id'].astype('int64')

df_train = df_train[df_train['url'].notna()] # use only url is not NaN
df_train.head()

,question,answer,question_id,answer_id,url,upvotes,answer_creation_time
0,怎么说服男朋友买烤箱？,emmmmm，首先想说的是，我买厨房用品一般是不用「说服」的，只是在厨房堆的满满当当的情况下...,357137111,914332816,https://www.zhihu.com/question/357137111/answe...,15,2019-11-28T12:01:22.000Z
1,航天从业者是如何看待电视剧《你是我的荣耀》的？,难得有个关于航天的剧，职场情节悬不悬浮，航天设定和细节走不走心？带着放大镜看了前18集，...,475169837,2053313113,https://www.zhihu.com/question/475169837/answe...,4432,2021-08-11T07:26:08.000Z
2,如何看待PayPal正式进入中国？,PayPal不仅是美国支付巨头，也是国际支付巨头，目前已开拓全球200多个市场，美国以外的市...,348551037,866103409,https://www.zhihu.com/question/348551037/answe...,127,2019-10-22T09:11:15.000Z
3,中金公司交易员月薪八万五是如何做到的？,1、首先，考虑到这位交易员的工作经验，月薪八万五的表述是不正确的：其实是一年的全部薪酬除以1...,545938899,2602363788,https://www.zhihu.com/question/545938899/answe...,450,2022-07-31T13:29:04.000Z
4,摇滚乐（金属）给你们带来了什么？,"ㄟ( ▔, ▔ )ㄏ哪里带来了什么东西啊，除了找到热爱的东西，也失去了很多。听重型现场像疯子...",361437216,1073541478,https://www.zhihu.com/question/361437216/answe...,5,2020-03-12T05:49:28.000Z


In [11]:
df_train_sorted = df_train.sort_values(by='upvotes', ascending=False).head(10000)
df_train_sorted.to_csv('../data/zhihu_kol_10k.csv', index=False)

# 30 examples to share

In [3]:
df_train_sorted = pd.read_csv('../data/zhihu_kol_10k.csv')
df_temp = df_train_sorted[['question', 'answer']].head(30)
df_temp.to_csv('../data/zhihu_kol_10k_sample.csv', index=False)

# Scrap question body

In [12]:
df_train_sorted = pd.read_csv('../data/zhihu_kol_10k.csv')

In [13]:
df_train_sorted.head(10)

,question,answer,question_id,answer_id,url,upvotes,answer_creation_time
0,长得太美是一种什么样的感觉？,我的亲妹妹，无论到哪儿都有人夸她好看，走在大街上都会被人要微信！！！ 去海南玩的时候随手拍了...,430297692,1715042961,https://www.zhihu.com/question/430297692/answe...,634000,2021-02-04T09:17:34.000Z
1,语文考试可以写负能量作文吗？,我真的是服你们这群崽子。 一个个的都是契科夫转世、莫泊桑临凡、欧亨利还魂，茨威格夺舍…… 你...,272068457,1302565494,https://www.zhihu.com/question/272068457/answe...,434000,2020-06-25T15:25:45.000Z
2,怎么在一个月内让英语听力有明显的提高？,相信我，看完这篇文章，一个月后你的英语听力水平将会超出你的想象。本篇也是知乎 2018、20...,24706380,387403508,https://www.zhihu.com/question/24706380/answer...,424000,2018-05-09T09:42:29.000Z
3,有哪些知识，你知道后你的世界马上就不一样了？,1.原则上可以，就是不可以；原则上不可以，就是可以！ 2.养成吃晚饭就洗碗的习惯，不要泡在水...,38632401,1060250796,https://www.zhihu.com/question/38632401/answer...,399000,2020-03-06T08:44:01.000Z
4,如何看透人的心理？,1、朋友圈经常晒自己自拍照的单身女生不能追。 2、“A+姓名+手机号”这种微信名，大多都...,365128026,1426528244,https://www.zhihu.com/question/365128026/answe...,363000,2020-08-22T10:32:08.000Z
5,有什么相见恨晚的背单词方法？,谢邀，单词，看这一篇就够了。全文手码5000字，被收藏750000次了，谢谢大家的支持。 我...,48040579,264770860,https://www.zhihu.com/question/48040579/answer...,291000,2017-11-24T01:03:59.000Z
6,你手机里最舍不得删的那张照片有什么故事?,这张纸条是我妻子写的。 写下的前24小时:由于已经晚于预产期一周，医生给用我妻子上了催产...,269575911,372282921,https://www.zhihu.com/question/269575911/answe...,273000,2018-04-21T03:41:38.000Z
7,人类有没有可能是被设计出来的？,如果是真的，那我一定要找到那个造物主，先把他按在地上一顿爆锤，然后厉声质问他：为什么要在我身...,298688205,528647866,https://www.zhihu.com/question/298688205/answe...,264000,2018-11-10T02:55:33.000Z
8,学霸的作息是怎样的，为什么都不会困？,北大15级的来答一发。 学霸不会困，靠的不是体力，是自我管理。 一天分成三个阶段，规律作息，...,267346432,426452693,https://www.zhihu.com/question/267346432/answe...,260000,2018-06-25T16:00:08.000Z
9,有哪些读书学不来，却很重要的素质？,（本文所有文字皆为原创，除注明引用外未参考任何文献，谢绝转载，） 书上找不到，也很少有人讨论...,28626263,41992632,https://www.zhihu.com/question/28626263/answer...,259000,2015-03-14T10:48:43.000Z


In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

# Initialize Selenium WebDriver
def initialize_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    
    service = Service(r"C:\Users\JeffreyCh0\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")  # Replace with the path to your ChromeDriver
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

# Scrape body content from URL
def scrape_body(driver, url):
    try:
        driver.get(url)
        time.sleep(2)  # Wait for the page to load (adjust as needed)
        
        # Modify the following line to target the specific part of the page you want to scrape
        body = driver.find_element(By.TAG_NAME, "body").text
        return body
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

# Main scraping function
def add_body_column(df):
    driver = initialize_driver()
    bodies = []
    for url in df['url']:
        print(f"Scraping: {url}")
        body = scrape_body(driver, url)
        bodies.append(body)
    driver.quit()
    df['body'] = bodies
    return df

In [35]:
df_temp['url'][0]

'https://www.zhihu.com/question/430297692/answer/1715042961'

In [37]:
# Example DataFrame

# Add 'body' column
df_temp = add_body_column(df_temp)

df_temp['body'][0]


Scraping: https://www.zhihu.com/question/430297692/answer/1715042961
Scraping: https://www.zhihu.com/question/272068457/answer/1302565494
Scraping: https://www.zhihu.com/question/24706380/answer/387403508


<ipython-input-32-60e7449e53fd>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['body'] = bodies


'进入知乎\n系统监测到您的网络环境存在异常，为保证您的正常访问，请点击下方验证按钮进行验证。在您验证完成前，该提示将多次出现。\n开始验证\n登录知乎\n·\n意见反馈'

# Prepare for IR

In [23]:
from datasets import load_dataset
ds = load_dataset("wangrui6/Zhihu-KOL")

data_dict = {}
data_dict['question'] = ds['train']['INSTRUCTION']
for key in json.loads(ds['train']['METADATA'][0]).keys():
    if key == "question_id":
        data_dict[key] = [int(json.loads(x)[key]) for x in ds['train']['METADATA']]

df_question = pd.DataFrame(data_dict)
del data_dict
df_question = df_question.drop_duplicates(subset=['question_id'])
print(df_question.shape[0])
df_question.head()

159007


,question,question_id
0,怎么说服男朋友买烤箱？,357137111
1,航天从业者是如何看待电视剧《你是我的荣耀》的？,475169837
2,如何看待PayPal正式进入中国？,348551037
3,中金公司交易员月薪八万五是如何做到的？,545938899
4,摇滚乐（金属）给你们带来了什么？,361437216


In [9]:
emb_model = BGEM3FlagModel('BAAI/bge-m3',  
                       use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [26]:
zh_sentences = df_question['question'].tolist()
zh_sentences = list(set(zh_sentences))

zh_emb = emb_model.encode(zh_sentences)['dense_vecs']

Inference Embeddings: 100%|██████████| 622/622 [06:11<00:00,  1.68it/s]


In [31]:
import pickle
with open('../data/zhihu_emb.pkl', 'wb') as f:
    pickle.dump((zh_sentences,zh_emb), f)

In [28]:
reddit = pd.read_csv('../data/reddit_sample.csv')
reddit.columns = ["question", "detail", "answer", "topic", "answer_length"]
reddit_sentences = reddit['question'].tolist()

reddit_emb = emb_model.encode(reddit_sentences)['dense_vecs']

similarity = reddit_emb @ zh_emb.T

top_k = 5
list_top_k = []
for i in range(similarity.shape[0]):
    top_k_idx = np.argsort(similarity[i])[::-1][:top_k]
    top_k_sim = similarity[i][top_k_idx]
    list_top_k.append([(sim,zh_sentences[j]) for sim, j in zip(top_k_sim,top_k_idx)])

reddit['top_1'] = [x[0][1] for x in list_top_k]
reddit['top_1_sim'] = [x[0][0] for x in list_top_k]
reddit['top_2'] = [x[1][1] for x in list_top_k]
reddit['top_2_sim'] = [x[1][0] for x in list_top_k]
reddit['top_3'] = [x[2][1] for x in list_top_k]
reddit['top_3_sim'] = [x[2][0] for x in list_top_k]
reddit['top_4'] = [x[3][1] for x in list_top_k]
reddit['top_4_sim'] = [x[3][0] for x in list_top_k]
reddit['top_5'] = [x[4][1] for x in list_top_k]
reddit['top_5_sim'] = [x[4][0] for x in list_top_k]

reddit.to_csv('../data/reddit_zhihu_matched_sample.csv', index=False)

In [30]:
reddit = pd.read_csv('../data/reddit_sample.csv')
reddit.columns = ["question", "detail", "answer", "topic", "answer_length"]
reddit_questions = reddit['question'].tolist()
reddit_details = reddit['detail'].tolist()
reddit_sentences = [str(x) + " " + str(y) for x, y in zip(reddit_questions, reddit_details)]


reddit_emb = emb_model.encode(reddit_sentences)['dense_vecs']

similarity = reddit_emb @ zh_emb.T

top_k = 5
list_top_k = []
for i in range(similarity.shape[0]):
    top_k_idx = np.argsort(similarity[i])[::-1][:top_k]
    top_k_sim = similarity[i][top_k_idx]
    list_top_k.append([(sim,zh_sentences[j]) for sim, j in zip(top_k_sim,top_k_idx)])

reddit['top_1'] = [x[0][1] for x in list_top_k]
reddit['top_1_sim'] = [x[0][0] for x in list_top_k]
reddit['top_2'] = [x[1][1] for x in list_top_k]
reddit['top_2_sim'] = [x[1][0] for x in list_top_k]
reddit['top_3'] = [x[2][1] for x in list_top_k]
reddit['top_3_sim'] = [x[2][0] for x in list_top_k]
reddit['top_4'] = [x[3][1] for x in list_top_k]
reddit['top_4_sim'] = [x[3][0] for x in list_top_k]
reddit['top_5'] = [x[4][1] for x in list_top_k]
reddit['top_5_sim'] = [x[4][0] for x in list_top_k]

reddit.to_csv('../data/reddit_zhihu_matched_sample_wDetail.csv', index=False)

# LLM annotation


In [6]:
import pandas as pd
from openai import OpenAI
import os
import json

In [7]:
reddit = pd.read_csv('../data/reddit_zhihu_matched_sample_wDetail.csv')

In [25]:
def ir_top5(question, candidates):
        
    client = OpenAI(api_key = os.environ['OPENAI_API_KEY'])
    user_prompt = f"# Question:\n{question}\n\n # Candidates:\nA. {candidates[0]}\nB. {candidates[1]}\nC. {candidates[2]}\nD. {candidates[3]}\nE. {candidates[4]}\n\n"

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
            "role": "system",
            "content": [
                {
                "type": "text",
                "text": "Given an English question, pick the most similar one from the list of 5 Chinese questions."
                }
            ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_prompt
                    }
                ]
            }
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
            "name": "similar_question_response",
            "strict": True,
            "schema": {
                "type": "object",
                "properties": {
                "response": {
                    "type": "string",
                    "description": "The letter corresponding to the most similar question.",
                    "enum": [
                    "A",
                    "B",
                    "C",
                    "D",
                    "E"
                    ]
                }
                },
                "required": [
                "response"
                ],
                "additionalProperties": False
            }
            }
        },
        temperature=1,
        max_completion_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )
    
    return json.loads(response.choices[0].message.content)["response"]

In [ ]:
reddit["gpt_pick"] = reddit.apply(lambda x: ir_top5(str(x['question'])+"\n"+str(x['detail']), [x['top_1'], x['top_2'], x['top_3'], x['top_4'], x['top_5']]), axis=1)
reddit["gpt_pick_question"] = reddit.apply(lambda x: x[f"top_{'_ABCDE'.index(x['gpt_pick'])}"], axis=1)

reddit.to_csv('../data/reddit_zhihu_matched_sample_wDetail.csv', index=False)

In [36]:
def sim_check(eng_q, chi_q):
        
    client = OpenAI(api_key = os.environ['OPENAI_API_KEY'])
    user_prompt = f"# English Question:\n{eng_q}\n\n # Chinese Question:\n{chi_q}\n\n"

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
            "role": "system",
            "content": [
                {
                "type": "text",
                "text": "Given an English question and a Chinese question, determine whether they are asking the same question."
                }
            ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_prompt
                    }
                ]
            }
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
            "name": "similar_question_response",
            "strict": True,
            "schema": {
                "type": "object",
                "properties": {
                "response": {
                    "type": "boolean",
                    "description": "Whether the English question and Chinese question are asking the same question.",
                }
                },
                "required": [
                "response"
                ],
                "additionalProperties": False
            }
            }
        },
        temperature=1,
        max_completion_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )
    
    return json.loads(response.choices[0].message.content)["response"]

In [37]:
reddit["gpt_sim"] = reddit.apply(lambda x: sim_check(str(x['question'])+"\n"+str(x['detail']), x['gpt_pick_question']), axis=1)

In [38]:
reddit.to_csv('../data/reddit_zhihu_matched_sample_wDetail.csv', index=False)

In [8]:
def ir_top5_alt(question, candidates):
        
    client = OpenAI(api_key = os.environ['OPENAI_API_KEY'])
    user_prompt = f"# Question:\n{question}\n\n # Candidates:\nA. {candidates[0]}\nB. {candidates[1]}\nC. {candidates[2]}\nD. {candidates[3]}\nE. {candidates[4]}\n\n"

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
            "role": "system",
            "content": [
                {
                "type": "text",
                "text": "Given an English question and a list of 5 Chinese questions, pick the Chinese question that asks the same question as the English question. If none of the Chinese questions ask the same question, select 'F'."
                }
            ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_prompt
                    }
                ]
            }
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
            "name": "similar_question_response",
            "strict": True,
            "schema": {
                "type": "object",
                "properties": {
                "response": {
                    "type": "string",
                    "description": "The letter corresponding to the most similar question. if none of the Chinese questions ask the same question, select 'F'.",
                    "enum": [
                    "A",
                    "B",
                    "C",
                    "D",
                    "E",
                    "F"
                    ]
                }
                },
                "required": [
                "response"
                ],
                "additionalProperties": False
            }
            }
        },
        temperature=1,
        max_completion_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )
    
    return json.loads(response.choices[0].message.content)["response"]

In [9]:
reddit["gpt_pick_alt"] = reddit.apply(lambda x: ir_top5_alt(str(x['question'])+"\n"+str(x['detail']), [x['top_1'], x['top_2'], x['top_3'], x['top_4'], x['top_5']]), axis=1)
reddit.to_csv('../data/reddit_zhihu_matched_sample_wDetail.csv', index=False)

# Expense calculator

In [1]:
import pandas as pd
import numpy as np

In [2]:
reddit = pd.read_csv('../data/reddit_zhihu_matched_sample_wDetail.csv')


In [4]:
import tiktoken

def count_tokens(text, model="gpt-4o-mini"):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# Example usage:
text = "Hello, how are you?"
print(count_tokens(text))  # Output: Token count

6


In [15]:
question_token_len = np.mean([count_tokens(str(x)) for x in reddit['question'].tolist()])
detail_token_len = np.mean([count_tokens(str(x)) for x in reddit['detail'].tolist()])
q1_token_len = np.mean([count_tokens(str(x)) for x in reddit['top_1'].tolist()])
q2_token_len = np.mean([count_tokens(str(x)) for x in reddit['top_2'].tolist()])
q3_token_len = np.mean([count_tokens(str(x)) for x in reddit['top_3'].tolist()])
q4_token_len = np.mean([count_tokens(str(x)) for x in reddit['top_4'].tolist()])
q5_token_len = np.mean([count_tokens(str(x)) for x in reddit['top_5'].tolist()])
gpt_pick_question_token_len = np.mean([count_tokens(str(x)) for x in reddit['gpt_pick_question'].tolist()])


In [25]:
# method 1
ir_top_5_token = question_token_len + detail_token_len + q1_token_len + q2_token_len + q3_token_len + q4_token_len + q5_token_len
sim_check_token = question_token_len + detail_token_len + gpt_pick_question_token_len

method_1_token = ir_top_5_token + sim_check_token

# method 2
method_2_token = ir_top_5_token

print("Method 1 token count: ", round(method_1_token))
print("Method 2 token count: ", round(method_2_token))

Method 1 token count:  640
Method 2 token count:  352


In [29]:
# gpt-4o-mini: $0.150 / 1M tokens
# gpt-4o:      $2.50 / 1M tokens
price1_mini = 28809*method_1_token*(0.15/1e6)
price1_4o = 28809*method_1_token*(2.5/1e6)
price2_mini = 28809*method_2_token*(0.15/1e6)
price2_4o = 28809*method_2_token*(2.5/1e6)

print(f"Method 1 cost: 4o - ${round(price1_4o, 2)}, 4o-mini - ${round(price1_mini, 2)}, effective outcome size: {round(28809*0.33)}, precision: 0.8")
print(f"Method 2 cost: 4o - ${round(price2_4o, 2)}, 4o-mini - ${round(price2_mini, 2)}, effective outcome size: {round(28809*0.6)}, precision: 0.61")

Method 1 cost: 4o - $46.06, 4o-mini - $2.76, effective outcome size: 9507, precision: 0.8
Method 2 cost: 4o - $25.36, 4o-mini - $1.52, effective outcome size: 17285, precision: 0.61
